In [1]:
%run ./ipy_setup.py
%matplotlib inline
#%run ../scripts/greedy_partitioner.py
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Graphics import GenomeDiagram
from reportlab.lib.units import cm

def translateContigName(cname):
    outstr = "%06dF" % (int(cname)-1)
    return outstr

In [265]:
xmap = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/alex_concat_hmaps_aligned.xmap', 
                   sep='\t', skiprows=8)
xmap = xmap.drop(xmap.index[0]) # first line after header is junk
mergemap = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/alex_merged_maps.txt', 
                   sep=' ')
hmap = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/alex_concat_hmaps.hmap', 
                   sep='\t', skiprows=9)
hmap = xmap.drop(xmap.index[0]) # first line after header is junk

cmap_q = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/alex_concat_hmaps_aligned_q.cmap', 
                   sep='\t', header=None, skiprows=10)
#cmap_q = cmap_q.drop(cmap_q.index[0]) # first line after header is junk
cmap_r = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/NIST_asm_2015-07-01_pa_ctg_BspQI_0Kb_0labels.cmap', 
                   sep='\t', header=None, skiprows=7)
#cmap_r = cmap_r.drop(cmap_r.index[0]) # first line after header is junk

xmap.RefContigID = xmap.RefContigID.astype(int)
xmap.RefStartPos = xmap.RefStartPos.astype(float)
xmap.RefEndPos = xmap.RefEndPos.astype(float)
xmap = xmap.sort(columns=['RefContigID','RefStartPos'])

block_reader = HapCutReader('/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_qv13_mq10/hg002_hapcut_000000F.hapcut')

blockpos = []
for i in block_reader.blocks:
    block = block_reader.loc(i)
    blockpos.append([i, block.chrom, block.start, block.end])
    
opticalmap_hapinform = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/alex_merged_hapinform_000000F.txt', sep=' ', 
                         header=None)
opticalmap_hapinform.sort(columns=5, inplace=True)

opticalmap_all_a = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine/alex_merged_maps.txt', sep=' ')
opticalmap_all_a = opticalmap_all_a[opticalmap_all_a['r_contig_id'] == 1]
#opticalmap_all.sort(columns='r_pos', inplace=True)
opticalmap_all = opticalmap_all_a[opticalmap_all_a['HapDelta'] != 0.0]
#opticalmap_all

vcf_matches = pd.read_csv('/hpc/users/neffr01/2work/alex_na24385_phasing/hg002_snps_opticalmap_matches_2_000000F.vcf', sep='\t', header=None)

In [285]:
gdd = GenomeDiagram.Diagram('XMap file')
gdt_features = gdd.new_track(1, greytrack=False, title="000000F")
gds_features = gdt_features.new_set()

#Add three features to show the strand options
state = 0
lastcontig = None
contigcount = 0
for a,i in xmap.iterrows():
    state += 1
    if i.RefContigID != 1:
        break
    #strand = +1 if i.Orientation == '+' else -1
    color = colors.lightgreen if state % 2 == 0 else colors.green
    feature = SeqFeature(FeatureLocation(int(i.RefStartPos), int(i.RefEndPos)), strand=+1)
    gds_features.add_feature(feature, name="q-" + i.QryContigID, label=True, color=color, 
                             label_position="middle", label_angle=0)#,
                         #sigil="ARROW", arrowhead_length=0.15)
for i in blockpos:
    state += 1
    color = colors.lightblue if state % 2 == 0 else colors.blue
    feature = SeqFeature(FeatureLocation(i[2], i[3], strand=-1))
    gds_features.add_feature(feature, name="b-" + str(i[0]), label=True, color=color, label_position="middle", 
                             label_angle=-180)
for a,i in opticalmap_all.iterrows():
    color = colors.yellow
    _,cmap_r_curr = cmap_r[(cmap_r[0] == 1) & (cmap_r[3] == i['r_site_id'])].iterrows().next()
    _,cmap_r_last = cmap_r[(cmap_r[0] == 1) & (cmap_r[3] == i['r_site_id']-1)].iterrows().next()
    #print cmap_r_curr, cmap_r_last
    feature = SeqFeature(FeatureLocation(int(cmap_r_last[5]), int(cmap_r_curr[5]), strand=0))
    gds_features.add_feature(feature, name="SV-" + str(int(i['r_site_id'])), label=True, color=color, label_position="middle", 
                             label_angle=0)

for a,i in opticalmap_hapinform.iterrows():
    color = colors.red
    feature = SeqFeature(FeatureLocation(int(i[5]), int(i[5])+10, strand=0))
    gds_features.add_feature(feature, name="var-" + str(int(i[4])), label=True, color=color, label_position="middle", 
                             label_angle=0)

for a,i in vcf_matches.iterrows():
    color = colors.springgreen
    feature = SeqFeature(FeatureLocation(int(i[1]), int(i[1])+50, strand=-1))
    gds_features.add_feature(feature, name="VCF-" + str(a), label=True, color=color, label_position="bottom", 
                             label_angle=0, sigil="ARROW", arrowhead_length=0.5)
    
gdd.draw(format='linear', pagesize=(30*cm,30*cm), fragments=20,
       start=0, end=33000000)
gdd.write("contig000000F.test.pdf", "pdf")

In [315]:
'''
interrogate:

block 3087 in hapcut blocks
optical variants 525 and 550
vcf variants line 2-4 from overlap file
optical contig q-20

'''

block = block_reader.loc(3087)
blockvars = [b for b in block.variants.values() if b.pos in (4207133, 4425224) ]
opticalvars = opticalmap_hapinform[[True if i in [525, 550] else False for i in opticalmap_hapinform[4]]]
vcfvars = vcf_matches[2:6:2]
print "optical variant line"
print opticalvars
print ""
print "vcf variant line"
print vcfvars
print ""
print "block variant line"
for v in blockvars:
    print v.chrom, v.pos, v.hap1, v.hap2


optical variant line
    0    1          2   3    4        5   6   7     8   9      10  11
13  20  111   858070.1   1  525  4207133   1   0  18.8  -1  35.68   0
14  20  133  1076495.5   1  550  4425218   2   0   4.7  -1  21.43   0

vcf variant line
         0        1  2  3  4       5  6                                                  7                        8                                                9
2  000000F  4207133  .  G  A  105.77  .  AC=1;AF=0.500;AN=2;BaseQRankSum=-0.165;DP=73;D...           GT:AD:DP:GQ:PL                        0/1:45,25:70:99:134,0,267
4  000000F  4425224  .  C  G  620.91  .  AB=0.52381;ABP=3.21711;AC=1;AF=0.5;AN=2;AO=22;...  GT:GQ:DP:RO:QR:AO:QA:GL  0/1:153.86:42:20:690:22:767:-68.4462,0,-61.5125

block variant line
000000F 4207133 1 0
000000F 4425224 0 1
